In [ ]:
#how data are represented at each level (forward, backward, forward with padding on top) needs a little
    #experimentation to determine the best representation
    #also, is encoding at each layer really the best way? or just feeding the raw through?
    
#Outside web ips are going to be a problem/messy/noisy. Start by categorizing all outside ips by <OUTSIDE_IP>
    #instead of the ip address, or another 4 digit symbol to insert into the hex string.
    
#to help the models generalize more, for a given source ip address with probability p (say p = 0.1) 
    #use the token <OTHER_MACHINE>
    
#should we remove random parts of the header, i.e. checksum

#should I take out bias for RNNs?

#for the decoder,does the fork encoding need to happen ?
    #do we simply cat the hContext with the next words?
    
#Should the architecture just be encode, context and then prediction???

#Input data, should it have character and hex pair encoding as well?

In [1]:
%matplotlib inline

import os
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=gpu'#,optimizer=fast_compile'

import cPickle
import sys
import binascii
import multiprocessing as mp
from itertools import chain
from scapy.all import *
#sys.path.append('hed-dlg/')
from collections import OrderedDict

import numpy as np
import random
from scipy.stats import itemfreq
import matplotlib.pyplot as plt
from copy import copy

import blocks
from blocks.bricks import Linear, Softmax, Softplus, NDimensionalSoftmax, BatchNormalizedMLP, \
                                Rectifier, Logistic, Tanh, MLP
from blocks.bricks.recurrent import GatedRecurrent, Fork, LSTM
from blocks.initialization import Constant, IsotropicGaussian, Identity, Uniform
from blocks.bricks.cost import BinaryCrossEntropy, CategoricalCrossEntropy
from blocks.filter import VariableFilter
from blocks.roles import PARAMETER
from blocks.graph import ComputationGraph

import theano
from theano import tensor as T

sys.setrecursionlimit(100000)
###These warnings do not impede progress
#WARNING: Failed to execute tcpdump. Check it is installed and in the PATH
#WARNING: No route found for IPv6 destination :: (no default route?)


Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, CuDNN 4007)


# Data

In [2]:
dataPath = '/data/fs4/datasets/pcaps/bigFlows.pcap'
#pcaps = rdpcap(dataPath)
#sessionPrep = pcaps.sessions()

In [3]:
import ast
import json
import subprocess
import sys

def parse_header(line):
    ret_dict = {}
    h = line.split()
    #ret_dict['direction'] = " ".join(h[3:6])

    if h[2] == 'IP6':
        """
        Conditional formatting based on ethernet type.
        IPv4 format: 0.0.0.0.port
        IPv6 format (one of many): 0:0:0:0:0:0.port
        """
        ret_dict['src_port'] = h[3].split('.')[-1]
        ret_dict['src_ip'] = h[3].split('.')[0]

        ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
        ret_dict['dest_ip'] = h[5].split('.')[0]
    else:
        if len(h[3].split('.')) > 4:
            ret_dict['src_port'] = h[3].split('.')[-1]
            ret_dict['src_ip'] = '.'.join(h[3].split('.')[:-1])
        else:
            ret_dict['src_ip'] = h[3]
            ret_dict['src_port'] = ''

        if len(h[5].split('.')) > 4:
            ret_dict['dest_port'] = h[5].split('.')[-1].split(':')[0]
            ret_dict['dest_ip'] = '.'.join(h[5].split('.')[:-1])
        else:
            ret_dict['dest_ip'] = h[5].split(':')[0]
            ret_dict['dest_port'] = ''

    try:
        if ret_dict['src_port'] == '53' or ret_dict['dst_port'] == '53':
            ret_dict['length'] = int(h[-1][1:-1])
        else:
            try:
                ret_dict['length'] = int(line.split(' length ')[1].split(':')[0])
            except:
                ret_dict['length'] = 0
    except:
        try:
            ret_dict['length'] = int(line.split(' length ')[1].split(':')[0])
        except:
            ret_dict['length'] = 0

    return ret_dict

def parse_data(line, length):
    ret_str = ''
    h, d = line.split(':', 1)
    ret_str = d.strip().replace(' ', '')
    if length != 0:
        ret_str = ret_str[:-(2 * length)]
    return ret_str

def process_packet(output):
    # TODO!! throws away the first packet!
    try:
        ret_header = {}
        ret_dict = {}
        ret_data = ''
        for line in output:
            line = line.strip()
            if line.startswith('0x'):
                data = parse_data(line, ret_header['length'])
                ret_data = ret_data + data
            else:
                ret_header = parse_header(line)
                ret_dict.update(ret_header)
                if ret_data != '':
                    ret_dict['data'] = ret_data
                    ret_data = ''
                yield ret_dict
    except:
        pass
    
def read_pcap(path):
    hex_sessions = {}
    proc = subprocess.Popen('tcpdump -nn -tttt -xx -r '+path,
                            shell=True,
                            stdout=subprocess.PIPE)
    for packet in process_packet(proc.stdout):
        a = str(packet)
    
        a = ast.literal_eval(a)
        if 'data' in a:
            key = (a['src_ip']+":"+a['src_port'], a['dest_ip']+":"+a['dest_port'])
            keys = [k for k in hex_sessions if (k[0] == key[0] and k[1] == key[1]) or (k[0] == key[1] and k[1] == key[0])]
            if len(keys) > 0:
                #hex_sessions[keys[0]].append(a['direction']+a['data'])
                hex_sessions[keys[0]].append(a['data'])
            else:
                #hex_sessions[key] = [a['direction']+a['data']]
                hex_sessions[key] = [a['data']]
    return hex_sessions

In [4]:
hexSessions = read_pcap(dataPath)

In [5]:
#get rid of any sessions with less than 80 characters
for ses in hexSessions.keys():
    paclens = []
    for pac in hexSessions[ses]:
        paclens.append(len(pac))
    if np.min(paclens)<80:
        del hexSessions[ses]

In [ ]:
#turns the sessions into a dictionary key = session_number, val = list of packages in hex
minPackets = 2

def hexSessionDictCreator(scapySessions, minPackets = 2):

    hexSessions = OrderedDict()
    i=0
    for k,v in sessionPrep.items(): # v is the session
        
        if len(v) < minPackets:
            pass
        
        else:
            scpcaps = []    
            
            for p in v: #p is the individual packet in the session  
                
                try: #getting rid of payload
                    rawindex = len(p[Raw]) 
                    #payloadLens.append(rawindex)
                    scpcaps.append(binascii.hexlify(str(p.original)[:-rawindex])) #turn it into hex
                
                except: #if no payload
                    scpcaps.append(binascii.hexlify(str(p.original)))
    
            hexSessions['session_' + str(i)] = scpcaps
            i+=1
    #assert that all sessions have len of at least minPackets
    return hexSessions

hexSessionstest = hexSessionDictCreator(sessionPrep, minPackets=minPackets)

In [6]:
#Making the hex dictionary
def hexTokenizer():
    hexstring = '0,	1,	2,	3,	4,	5,	6,	7,	8,	9,	A,	B,	C,	D,	E,	F,	10,	11,	12,	13,	14,	15,	16,	17,	18,	19\
    ,	1A,	1B,	1C,	1D,	1E,	1F,	20,	21,	22,	23,	24,	25,	26,	27,	28,	29,	2A,	2B,	2C,	2D,	2E,	2F,	30,	31,	32,	33,	34,	35\
    ,	36,	37,	38,	39,	3A,	3B,	3C,	3D,	3E,	3F,	40,	41,	42,	43,	44,	45,	46,	47,	48,	49,	4A,	4B,	4C,	4D,	4E,	4F,	50,	51\
    ,	52,	53,	54,	55,	56,	57,	58,	59,	5A,	5B,	5C,	5D,	5E,	5F,	60,	61,	62,	63,	64,	65,	66,	67,	68,	69,	6A,	6B,	6C,	6D\
    ,	6E,	6F,	70,	71,	72,	73,	74,	75,	76,	77,	78,	79,	7A,	7B,	7C,	7D,	7E,	7F,	80,	81,	82,	83,	84,	85,	86,	87,	88,	89\
    ,	8A,	8B,	8C,	8D,	8E,	8F,	90,	91,	92,	93,	94,	95,	96,	97,	98,	99,	9A,	9B,	9C,	9D,	9E,	9F,	A0,	A1,	A2,	A3,	A4,	A5\
    ,	A6,	A7,	A8,	A9,	AA,	AB,	AC,	AD,	AE,	AF,	B0,	B1,	B2,	B3,	B4,	B5,	B6,	B7,	B8,	B9,	BA,	BB,	BC,	BD,	BE,	BF,	C0,	C1\
    ,	C2,	C3,	C4,	C5,	C6,	C7,	C8,	C9,	CA,	CB,	CC,	CD,	CE,	CF,	D0,	D1,	D2,	D3,	D4,	D5,	D6,	D7,	D8,	D9,	DA,	DB,	DC,	DD\
    ,	DE,	DF,	E0,	E1,	E2,	E3,	E4,	E5,	E6,	E7,	E8,	E9,	EA,	EB,	EC,	ED,	EE,	EF,	F0,	F1,	F2,	F3,	F4,	F5,	F6,	F7,	F8,	F9\
    ,	FA,	FB,	FC,	FD,	FE,	FF'.replace('\t', '')

    hexList = [x.strip() for x in hexstring.lower().split(',')]
    hexList.append('<EOP>') #End Of Packet token
    #EOS token??????
    hexDict = {}

    for key, val in enumerate(hexList):
        if len(val) == 1:
            val = '0'+val
        hexDict[val] = key  #dictionary k=hex, v=int  
    
    return hexDict

In [7]:
hexDict = hexTokenizer()

# Dictionary of IP communications

In [8]:
def srcIpDict(hexSessionDict):
    ''' 
    input: dictionary of key = sessions, value = list of HEX HEADERS of packets in session
    output: dictionary of key = source IP, value/subkey = dictionary of destination IPs, 
                                           subvalue = [[sport], [dport], [plen], [protocol]]
    
    '''
    
    srcIpDict = OrderedDict()   
    uniqIPs = [] #some ips are dest only. this will collect all ips, not just srcIpDict.keys()
    
    for session in hexSessionDict.keys():
        
        for rawpacket in hexSessionDict[session]:
            packet = copy(rawpacket)
            
            dstIpSubDict = {}

            srcip = packet[52:60]
            dstip = packet[60:68]
            sport = packet[68:72]
            dport = packet[72:76]
            plen = packet[32:36]
            protocol = packet[46:48]
            
            uniqIPs = list(set(uniqIPs) | set([dstip, srcip]))

            if srcip not in srcIpDict:
                dstIpSubDict[dstip] = [[sport], [dport], [plen], [protocol]]
                srcIpDict[srcip] = dstIpSubDict

            if dstip not in srcIpDict[srcip]:    
                srcIpDict[srcip][dstip] = [[sport], [dport], [plen], [protocol]]
            else:
                srcIpDict[srcip][dstip][0].append(sport)
                srcIpDict[srcip][dstip][1].append(dport)
                srcIpDict[srcip][dstip][2].append(plen)
                srcIpDict[srcip][dstip][3].append(protocol)

    return srcIpDict, uniqIPs

In [9]:
def dictUniquerizer(dictOdictsOlistOlists):
    '''
    input: dictionary of dictionaries that have a list of lists 
           ex. srcIpDict[srcip][dstip] = [[sport], [dport], [plen], [protocol]]
    output: dictionary of dictionaries with list of lists with unique items in the final sublist
    
    WARNING: will overwrite your input dictionary. Make a copy if you want to preserve dictOdictsOlistOlists.
    '''
    #dictCopy
    for key in dictOdictsOlistOlists.keys():
        for subkey in dictOdictsOlistOlists[key].keys():
            for sublist in xrange(len(dictOdictsOlistOlists[key][subkey])):
                dictOdictsOlistOlists[key][subkey][sublist] = list(set(dictOdictsOlistOlists[key][subkey][sublist]))
    
    return dictOdictsOlistOlists

In [10]:
#TODO: parallelize
#comsDicttest, uniqIPstest = srcIpDict(hexSessionstest)
comsDict, uniqIPs = srcIpDict(hexSessions)
#test
comsDict = dictUniquerizer(comsDict)
#comsDicttest = dictUniquerizer(comsDicttest)

# Adversarial Functions

### Directionality adversary

In [11]:
def ipDirSwitcher(hexSessionList):
    '''
    input is a list of packets from ONE session
    '''
    
    ipdirsession = []
        
    for p in hexSessionList:
        sourceIP = p[52:60]
        destIP = p[60:68]

        ipdirsession.append(p[:52]+destIP+sourceIP+p[68:])

    return ipdirsession

In [12]:
def portDirSwitcher(hexSessionList):
    '''
    input is a list of packets from ONE session
    '''
    
    portdirsession = []
    
    for p in hexSessionList:
        sport = p[68:72]
        dport = p[72:76]

        portdirsession.append(p[:68]+dport+sport+p[76:])

    return portdirsession

### Leave one swap one

In [13]:
def dstIpSwapOut(hexSessionList, dictOcoms, listOuniqIPs):
    #srcIpDict[srcip][dstip] = [[sport], [dport], [plen], [protocol]]
    
    swapSession = []
    srcip = hexSessionList[0][52:60] #assumes first packet contains true initial direction
    dstip = hexSessionList[0][60:68]
    normDstIps = dictOcoms[srcip].keys()+[srcip] #get list of dstIPs that srcIP talks to
    abbynormIps = copy(listOuniqIPs)
    
    for normIp in normDstIps:
        abbynormIps.remove(normIp) #remove itself and know dstIPs from list of consideration.
    
    abbynormDestIp = random.sample(abbynormIps, 1)[0] #get random ip that srcip doesn't talk to

    for rawpacket in hexSessionList:
        packet = copy(rawpacket)
        
        if packet[60:68] == dstip:
            packet = packet[:60] + abbynormDestIp + packet[68:] #
        elif packet[61:69] == srcip:
            packet = packet[:52] + abbynormDestIp + packet[60:] #in case direction switches for packet in session
            
        swapSession.append(packet)

    return swapSession
    

In [ ]:
def dstPortSwapOneOut(hexSessionList):
    #THINK THROUGH
    
    
    
    
    

In [14]:
def oneHot(index, granular = 'hex'):
    if granular == 'hex':
        vecLen = 257
    else:
        vecLen = 17
    
    zeroVec = np.zeros(vecLen)
    zeroVec[index] = 1.0
    
    return zeroVec
 

def oneSessionTokenized(sessionDict, hexDict, packetTimeSteps=50):

    dstMac = []
    srcMac = []
    firstType = []
    ihl = []
    tos = []
    firLen = []
    firId = []
    frag = []
    ttl = []
    protocol = []
    firCheck = []
    srcip = []
    dstip = []
    sport = []
    dport = []
    
    i=0
    for key in sessionDict.keys():
        sessionPackets = sessionDict[key]
        
        #if len(sessionPackets) > maxPackets: #crop the number of sessions to maxPackets
        #    sessionList = sessionPackets[:maxPackets]
        #else:
        #    sessionList = sessionPackets

        for packet in sessionPackets:
            
            dstMac.append(packet[0:12]) 
            srcMac.append(packet[12:24]) 
            firstType.append(packet[24:28])
            ihl.append(packet[28:30]) 
            tos.append(packet[30:32])
            firLen.append(packet[32:36])
            firId.append(packet[36:40]) 
            frag.append(packet[40:44]) 
            ttl.append(packet[44:46])
            protocol.append(packet[46:48])
            firCheck.append(packet[48:52]) 
            srcip.append(packet[52:60])
            dstip.append(packet[60:68])
            sport.append(packet[68:72])
            dport.append(packet[72:76])
            
            i+=1
    print i
    return np.asarray(dstMac), np.asarray(srcMac),np.asarray(firstType),np.asarray(ihl),np.asarray(tos),\
           np.asarray(firLen),np.asarray(firId),np.asarray(frag),np.asarray(ttl),np.asarray(protocol),\
           np.asarray(firCheck),np.asarray(srcip),np.asarray(dstip), np.asarray(sport), np.asarray(dport)

netstats  = oneSessionTokenized(hexSessions, hexDict)

indx = ['dstMac', 'srcMac', 'firstType', 'ihl', 'tos', 'firLen', 'firId', 'frag', 'ttl', 'protocol', 'firCheck',
       'srcip', 'dstip', 'sport', 'dport']
for num in range(13):
    print indx[num], len(np.unique(netstats[num]))


packet = hexSessions[hexSessions.keys()[1]][1]

coll = []
for key in hexSessions.keys():
    sessionPackets = hexSessions[key]
    for packet in sessionPackets:
        coll.append(packet[32:36]+packet[44:46]+packet[46:48]+packet[52:60]+packet[60:68]+packet[68:70]+packet[70:72]+
                   packet[72:74])

np.unique(coll).shape

In [15]:
maxPackets = 2
packetTimeSteps = 16

def oneSessionEncoder(sessionPackets, hexDict, maxPackets = 2, packetTimeSteps = 100,
                       packetReverse = False, charLevel = False, padOldTimeSteps = True):    
            
    sessionCollect = []
    packetCollect = []
    
    if charLevel:
        vecLen = 17
    else:
        vecLen = 257
    
    if len(sessionPackets) > maxPackets: #crop the number of sessions to maxPackets
        sessionList = copy(sessionPackets[:maxPackets])
    else:
        sessionList = copy(sessionPackets)

    for rawpacket in sessionList:
        packet = copy(rawpacket)
        packet = packet[32:36]+packet[44:46]+packet[46:48]+packet[52:60]+packet[60:68]+\
                 packet[68:70]+packet[70:72]+packet[72:74]
        packet = [hexDict[packet[i:i+2]] for i in xrange(0,len(packet)-2+1,2)]
        
        #print np.asarray(packet)
            
        if len(packet) >= packetTimeSteps: #crop packet to length packetTimeSteps
            packet = packet[:packetTimeSteps]
            packet = packet+[256] #add <EOP> end of packet token
        else:
            packet = packet+[256] #add <EOP> end of packet token
        
        packetCollect.append(packet)
        
        pacMat = np.array([oneHot(x) for x in packet]) #one hot encoding of packet into a matrix
        pacMatLen = len(pacMat)
        
        #padding packet
        if packetReverse:
            pacMat = pacMat[::-1]

        if pacMatLen < packetTimeSteps:
            #pad by stacking zeros on top of data so that earlier timesteps do not have information
            #padding the packet such that zeros are after the actual info for better translation
            if padOldTimeSteps:
                pacMat = np.vstack( ( np.zeros((packetTimeSteps-pacMatLen,vecLen)), pacMat) ) 
            else:
                pacMat = np.vstack( (pacMat, np.zeros((packetTimeSteps-pacMatLen,vecLen))) ) 

        if pacMatLen > packetTimeSteps:
            pacMat = pacMat[:packetTimeSteps, :]

        sessionCollect.append(pacMat)

    #padding session
    sessionCollect = np.asarray(sessionCollect, dtype=theano.config.floatX)
    numPacketsInSession = sessionCollect.shape[0]
    if numPacketsInSession < maxPackets:
        #pad sessions to fit the 
        sessionCollect = np.vstack( (sessionCollect,np.zeros((maxPackets-numPacketsInSession, 
                                                             packetTimeSteps, vecLen))) )
    
    return sessionCollect, packetCollect

# Learning functions

In [16]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def dropout(X, p=0.):
    if p != 0:
        retain_prob = 1 - p
        X = X / retain_prob * srng.binomial(X.shape, p=retain_prob, dtype=theano.config.floatX)
    return X

# Gradient clipping
def clip_norm(g, c, n): 
    '''n is the norm, c is the threashold, and g is the gradient'''
    
    if c > 0: 
        g = T.switch(T.ge(n, c), g*c/n, g) 
    return g

def clip_norms(gs, c):
    norm = T.sqrt(sum([T.sum(g**2) for g in gs]))
    return [clip_norm(g, c, norm) for g in gs]

# Regularizers
def max_norm(p, maxnorm = 0.):
    if maxnorm > 0:
        norms = T.sqrt(T.sum(T.sqr(p), axis=0))
        desired = T.clip(norms, 0, maxnorm)
        p = p * (desired/ (1e-7 + norms))
    return p

def gradient_regularize(p, g, l1 = 0., l2 = 0.):
    g += p * l2
    g += T.sgn(p) * l1
    return g

def weight_regularize(p, maxnorm = 0.):
    p = max_norm(p, maxnorm)
    return p

def Adam(params, cost, lr=0.0002, b1=0.1, b2=0.001, e=1e-8, l1 = 0., l2 = 0., maxnorm = 0., c = 8):
    
    updates = []
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    
    i = theano.shared(floatX(0.))
    i_t = i + 1.
    fix1 = 1. - b1**(i_t)
    fix2 = 1. - b2**(i_t)
    lr_t = lr * (T.sqrt(fix2) / fix1)
    
    for p, g in zip(params, grads):
        m = theano.shared(p.get_value() * 0.)
        v = theano.shared(p.get_value() * 0.)
        m_t = (b1 * g) + ((1. - b1) * m)
        v_t = (b2 * T.sqr(g)) + ((1. - b2) * v)
        g_t = m_t / (T.sqrt(v_t) + e)
        g_t = gradient_regularize(p, g_t, l1=l1, l2=l2)
        p_t = p - (lr_t * g_t)
        p_t = weight_regularize(p_t, maxnorm=maxnorm)
        
        updates.append((m, m_t))
        updates.append((v, v_t))
        updates.append((p, p_t))
    
    updates.append((i, i_t))
    #if iteration%100 == 0:
    #    updates.append((lr, lr*0.93))
    #else:
    #    updates.append((lr, lr))
    
    return updates

def RMSprop(cost, params, lr = 0.001, l1 = 0., l2 = 0., maxnorm = 0., rho=0.9, epsilon=1e-6, c = 8):
    
    grads = T.grad(cost, params)
    grads = clip_norms(grads, c)
    updates = []
    
    for p, g in zip(params, grads):
        g = gradient_regularize(p, g, l1 = l1, l2 = l2)
        acc = theano.shared(p.get_value() * 0.)
        acc_new = rho * acc + (1 - rho) * g ** 2
        updates.append((acc, acc_new))
        
        updated_p = p - lr * (g / T.sqrt(acc_new + epsilon))
        updated_p = weight_regularize(updated_p, maxnorm = maxnorm)
        updates.append((p, updated_p))
    return updates

In [ ]:
#makes output by shifting inputs down in time one step and then copying the last time step to the end.
#def targetModifier(targetArray):
#    newTarget = np.vstack((targetArray[1:, :], targetArray[-1,:]))
#    return newTarget

#def targetMaker(listOinputs):
    #TODO: do this with arrays
#    outputs = []
#    for inp in listOinputs:
#        outputs.append(targetModifier(inp))
#    outputs = np.asarray(outputs)
#    
#    return outputs

# Unsupervised feature extractor

### Initialization for both the unsupervised net and the classifier

In [182]:
X = T.tensor4('inputs')
Y = T.matrix('targets')

wtstd = 0.3
dimIn = 257 #hex has 256 characters + the <EOP> character
dim = 50 #dimension reduction size
rnnType = 'gru' #gru or lstm
bidirectional = False
linewt_init = IsotropicGaussian(wtstd)
line_bias = Constant(1.0)
rnnwt_init = IsotropicGaussian(wtstd)
rnnbias_init = Constant(0.0)
packetReverse = False

###ENCODER
if rnnType == 'gru':
    rnn = GatedRecurrent(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'gru')
    dimMultiplier = 2
else:
    rnn = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstm')
    dimMultiplier = 4

fork = Fork(output_names=['linear', 'gates'],
            name='fork', input_dim=dimIn, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)


###CONTEXT
if rnnType == 'gru':
    rnnContext = GatedRecurrent(dim=dim, weights_init = rnnwt_init, 
                                biases_init = rnnbias_init, name = 'gruContext')
else:
    rnnContext = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init, 
                      name = 'lstmContext')

forkContext = Fork(output_names=['linearContext', 'gatesContext'],
            name='forkContext', input_dim=dim, output_dims=[dim, dim * dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

if bidirectional:
    dimDec = dim*2
    
    if rnnType == 'gru':
        rnnContextRev = GatedRecurrent(dim=dim, weights_init = rnnwt_init, 
                                       biases_init = rnnbias_init, name = 'gruContextRev')
        
    else:
        rnnContextRev = LSTM(dim=dim, weights_init = rnnwt_init, biases_init = rnnbias_init,
                             name = 'lstmContextRev')
    
    rnnContextRev.initialize()

else:
    dimDec = dim


###DECODER
'''if rnnType == 'gru':
    rnnDec = GatedRecurrent(dim=dimIn, weights_init = rnnwt_init, 
                            biases_init = rnnbias_init, name = 'gruDecoder')
else:
    rnnDec = LSTM(dim=dimIn, weights_init = rnnwt_init, biases_init = rnnbias_init, name = 'lstmDecoder')'''


forkDec = Fork(output_names=['linear', 'gates'],
            name='forkDec', input_dim=dimDec, output_dims=[dim, dim*dimMultiplier], 
            weights_init = linewt_init, biases_init = line_bias)

#forkFinal = Fork(output_names=['linear', 'gates'],
#            name='forkFinal', input_dim=dim, output_dims=[dimIn, dimIn*dimMultiplier], 
#            weights_init = linewt_init, biases_init = line_bias)

#initialize the weights in all the functions
fork.initialize()
rnn.initialize()

forkContext.initialize()
rnnContext.initialize()
forkDec.initialize()

#forkFinal.initialize()
#rnnDec.initialize()

# Classifier

### Classifier graph

In [184]:
#Can be used with or without unsupervised learning

#X = T.matrix('classtest')
#Y = T.matrix('targets')
batch_sizeClass = 20
numClasses = 4
clippings = 1
learning_rateClass = theano.shared(np.array(0.001, dtype=theano.config.floatX))
learning_decay = np.array(0.9, dtype=theano.config.floatX)
classifierWts = IsotropicGaussian(wtstd)

bmlp = BatchNormalizedMLP(activations=[Logistic(),Logistic()], 
           dims=[dim, dim, numClasses],
           weights_init=classifierWts)


bmlp.initialize()

In [ ]:
def onestepEnc(X):
    data1, data2 = fork.apply(X) 

    if rnnType == 'gru':
        hEnc = rnn.apply(data1, data2) 
    else:
        hEnc, _ = rnn.apply(data2)
        #hinit, _ = rnn.apply(data2)
        #hEnc = hinit[:,-1] #hEnc shape = (batch_size, maxPackets, dim) 

    #hEnc2 = hEnc[-1]#T.reshape(hEnc[:,-1],(batch_sizeClass, maxPackets, dim))#the [:,-1] gets the last hidden state for 
                                                                    #each obs in minibatch
                                                                    #i.e. the last state for each sentence
    return hEnc

hEnc, _ = theano.scan(onestepEnc, X)
hEncReshape = T.reshape(hEnc[:,-1], (batch_sizeClass,maxPackets,1,dim))

def onestepContext(hEncReshape):
    
    data3, data4 = forkContext.apply(hEncReshape)
    
    if rnnType == 'gru':
        hContext = rnnContext.apply(data3, data4)
    else:
        hinitContext, _ = rnnContext.apply(data4)
        hContext = hinitContext #hContext shape = (batch_size, maxPackets, dim)
    
    return hContext

hContext, _ = theano.scan(onestepContext, hEncReshape)
hContextReshape = T.reshape(hContext[:,-1], (batch_sizeClass,dim))

data5, _ = forkDec.apply(hContextReshape)

pyx = bmlp.apply(data5)
softmax = Softmax()
softoutClass = softmax.apply(pyx)
costClass = T.mean(CategoricalCrossEntropy().apply(Y, softoutClass))

cgClass = ComputationGraph([costClass])
paramsClass = VariableFilter(roles = [PARAMETER])(cgClass.variables)

updatesClass = Adam(paramsClass, costClass, learning_rateClass, c=clippings) 
#updatesClass = RMSprop(costClass, paramsClass, learning_rateClass, c=clippings)

#print 'grad compiling'
#gradients = T.grad(costClass, paramsClass)
#gradients = clip_norms(gradients, clippings)
#gradientFun = theano.function([X,Y], gradients, allow_input_downcast=True)
#print 'finish with grads'

print 'compiling functions you talented soul'
classifierTrain = theano.function([X,Y], [costClass, hEnc, hContext, pyx, softoutClass], 
                                  updates=updatesClass, allow_input_downcast=True)
classifierPredict = theano.function([X], softoutClass, allow_input_downcast=True)
print 'finished compiling'

compiling functions you talented soul


### Classifier training

In [ ]:
def pickleModel(trainedClassifier, file2save2 = None, filePath='/work/notebooks/drawModels/', fileName = 'myModels'):
    
    if file2save2 == None:
        f=file(filePath+fileName+'.save', 'wb')
    else:
        f=file(filePath+file2save2, 'wb')
        
    cPickle.dump(trainedClassifier, f, protocol=cPickle.HIGHEST_PROTOCOL)

    f.close()
    
def loadModel(filePath):
    file2open = file(filePath, 'rb')
    model = cPickle.load(file2open)
    file2open.close()
    
    return model

In [ ]:
#randomize data, if not already done in unsupervised portion
hexSessionsKeys = hexSessions.keys()
#random.shuffle(hexSessionsKeys)
trainPercent = 0.9
trainIndex = int(len(hexSessionsKeys)*trainPercent)
padOldTimeSteps = True

runname = 'hredClassify'
epochCost = []
gradNorms = []
trainAcc = []
testAcc = []
#learning_rateClass.set_value(np.array(0.001, dtype=theano.config.floatX))

epochs = 200
iteration = 0

#normalTarget = np.array([0,1], dtype=theano.config.floatX)
#abbyTarget = np.array([1,0], dtype=theano.config.floatX)
costCollect = []
trainCollect = []
#epoch

for epoch in xrange(epochs):
    
    
    #iteration/minibatch
    for start, end in zip(range(0, trainIndex,batch_sizeClass),
                          range(batch_sizeClass, trainIndex, batch_sizeClass)):
        
        trainingTargets = []
        trainingSessions = []
        
        #create one minibatch with 0.5 normal and 0.5 abby normal traffic
        for trainKey in range(start, end):
            sessionForEncoding = list(hexSessions[hexSessions.keys()[trainKey]])
            
            #encode a normal session
            #oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
            #                              padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
            #                              packetTimeSteps = packetTimeSteps)
            #trainingSessions.append(oneHotSes[0])
            #trainingTargets.append(normalTarget)
            
            #encode an abby normal session
            adversaryList = [sessionForEncoding, 
                             dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs), 
                             portDirSwitcher(sessionForEncoding), 
                             ipDirSwitcher(sessionForEncoding)]
            abbyIndex = random.sample(range(len(adversaryList)), 1)[0]
            #abbyHexSession = random.sample(range(len(adversaryList), 1)[0]
            #abbyHexSession = ipDirSwitcher(sessionForEncoding)
            #abbyHexSession = dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs)
            #abbyHexSession = portDirSwitcher(sessionForEncoding)
            abbyOneHotSes = oneSessionEncoder(adversaryList[abbyIndex],
                                              hexDict = hexDict,
                                              packetReverse=packetReverse, 
                                              padOldTimeSteps = padOldTimeSteps, 
                                              maxPackets = maxPackets, 
                                              packetTimeSteps = packetTimeSteps)
            
            targetClasses = [0]*numClasses
            targetClasses[abbyIndex] = 1
            abbyTarget = np.array(targetClasses, dtype=theano.config.floatX)
            trainingSessions.append(abbyOneHotSes[0])
            trainingTargets.append(abbyTarget)
            
        sessionsMinibatch = np.asarray(trainingSessions).reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, dimIn))
        targetsMinibatch = np.asarray(trainingTargets)
    
        costfun = classifierTrain(sessionsMinibatch, targetsMinibatch)
        
        costCollect.append(costfun[0])
        trainCollect.append(np.mean(np.argmax(costfun[-1],axis=1) == np.argmax(targetsMinibatch, axis=1)))

        iteration+=1
        
        if iteration == 1:
            print 'you are amazing'
            
        if iteration%100 == 0:
            learning_rateClass.set_value(learning_rateClass.get_value() * learning_decay)
        
        if iteration%200 == 0:
            print
            print '   Iteration: ', iteration
            print '   Cost: ', np.mean(costCollect[-20:])
            print '   TRAIN accuracy: ', np.mean(trainCollect[-20:])
            print
            
            
            #print costfun[-1]
            #grads = gradientFun(sessionsMinibatch, targetsMinibatch)
            #for gra in grads:
            #    print '  gradient norms: ', np.linalg.norm(gra)
            np.savetxt('/data/fs4/home/bradh/outputs/'+runname+"_TRAIN.csv", trainCollect[::30], delimiter=",")
            np.savetxt('/data/fs4/home/bradh/outputs/'+runname+"_COST.csv", costCollect[::30], delimiter=",")

        #testing accuracy
        if iteration%500 == 0:
            testCollect = []

            #TODO: get accuracy for each adversarial type when they are all part of the training 
            #for ad in adversaryList:

            for start, end in zip(range(trainIndex, len(hexSessionsKeys), batch_sizeClass),
                                  range(trainIndex + batch_sizeClass, len(hexSessionsKeys), batch_sizeClass)):
                trainingTargets = []
                trainingSessions = []
                for trainKey in range(start, end):
                    sessionForEncoding = list(hexSessions[hexSessions.keys()[trainKey]])

                    #ENCODE NORMAL SESSION
                    #oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
                    #                              padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
                    #                              packetTimeSteps = packetTimeSteps)
                    #trainingSessions.append(oneHotSes[0])
                    #trainingTargets.append(normalTarget)

                    #ENCODE ABBY NORMAL SESSION
                    adversaryList = [sessionForEncoding, 
                                     dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs), 
                                     portDirSwitcher(sessionForEncoding), 
                                     ipDirSwitcher(sessionForEncoding)]
                    abbyIndex = random.sample(range(len(adversaryList)), 1)[0]
                    #abbyHexSession = random.sample(range(len(adversaryList), 1)[0]
                    #abbyHexSession = ipDirSwitcher(sessionForEncoding)
                    #abbyHexSession = dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs)
                    #abbyHexSession = portDirSwitcher(sessionForEncoding)
                    abbyOneHotSes = oneSessionEncoder(adversaryList[abbyIndex],
                                                      hexDict = hexDict,
                                                      packetReverse=packetReverse, 
                                                      padOldTimeSteps = padOldTimeSteps, 
                                                      maxPackets = maxPackets, 
                                                      packetTimeSteps = packetTimeSteps)

                    targetClasses = [0]*numClasses
                    targetClasses[abbyIndex] = 1
                    abbyTarget = np.array(targetClasses, dtype=theano.config.floatX)
                    trainingSessions.append(abbyOneHotSes[0])
                    trainingTargets.append(abbyTarget)

                #trainingSessions = [item for sublist in trainingSessions for item in sublist] #FIX in oneSessionEncoder
                sessionsMinibatch = np.asarray(trainingSessions, dtype=theano.config.floatX)\
                                               .reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, dimIn))
                targetsMinibatch = np.asarray(trainingTargets, dtype=theano.config.floatX)

                predcostfun = classifierPredict(sessionsMinibatch)
                testCollect.append(np.mean(np.argmax(predcostfun,axis=1) == np.argmax(targetsMinibatch, axis=1)))

            testAcc.append(np.mean(testCollect))
            np.savetxt('/data/fs4/home/bradh/outputs/'+runname+"_TEST.csv", testAcc, delimiter=",")

            #print 'PortSwitching accruacy: ', 
            #print 'IpSwitching accuracy: ', 
            #print 'IpSwap accuracy: ', 
            print 'TEST accuracy:         ', np.mean(testCollect)        
            print

            #save the models
            '''if iteration%1000 == 0:
                pickleModel(classifierTrain, filePath='/data/fs4/home/bradh/outputs/',
                            fileName=runname+'TRAIN'+str(iteration))
                pickleModel(classifierPredict, filePath='/data/fs4/home/bradh/outputs/',
                            fileName=runname+'PREDICT'+str(iteration)) '''
    
    epochCost.append(np.mean(costCollect[-50:]))
    trainAcc.append(np.mean(trainCollect[-50:]))
    print 'Epoch: ', epoch
    print 'Epoch cost average: ', epochCost[-1]
    print 'Epoch TRAIN accuracy: ', trainAcc[-1]


In [162]:
def predictClass(predictFun, hexSessionsDict, comsDict, uniqIPs, hexDict,
                 numClasses = 4, trainPercent = 0.9, dimIn=257, maxPackets=2,
                 packetTimeSteps = 16, padOldTimeSteps=True, batch_sizeClass = 20):
    
    
    predtargets = []
    actualtargets = []
    hexSessionsKeys = hexSessionsDict.keys()
    trainPercent = 0.9
    trainIndex = int(len(hexSessionsKeys)*trainPercent)
        
    for start, end in zip(range(trainIndex, len(hexSessionsKeys), batch_sizeClass),
                          range(trainIndex + batch_sizeClass, len(hexSessionsKeys), batch_sizeClass)):
        trainingSessions = []
        trainingTargets = []
        
        for trainKey in range(start, end):
            sessionForEncoding = list(hexSessionsDict[hexSessionsKeys[trainKey]])

            #ENCODE NORMAL SESSION
            #oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
            #                              padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
            #                              packetTimeSteps = packetTimeSteps)
            #trainingSessions.append(oneHotSes[0])
            #trainingTargets.append(normalTarget)

            #ENCODE ABBY NORMAL SESSION
            adversaryList = [sessionForEncoding, 
                             dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs), 
                             portDirSwitcher(sessionForEncoding), 
                             ipDirSwitcher(sessionForEncoding)]
            abbyIndex = random.sample(range(len(adversaryList)), 1)[0]
            #abbyHexSession = random.sample(range(len(adversaryList), 1)[0]
            #abbyHexSession = ipDirSwitcher(sessionForEncoding)
            #abbyHexSession = dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs)
            #abbyHexSession = portDirSwitcher(sessionForEncoding)
            abbyOneHotSes = oneSessionEncoder(adversaryList[abbyIndex],
                                              hexDict = hexDict,
                                              packetReverse=packetReverse, 
                                              padOldTimeSteps = padOldTimeSteps, 
                                              maxPackets = maxPackets, 
                                              packetTimeSteps = packetTimeSteps)

            targetClasses = [0]*numClasses
            targetClasses[abbyIndex] = 1
            abbyTarget = np.array(targetClasses, dtype=theano.config.floatX)
            trainingSessions.append(abbyOneHotSes[0])
            trainingTargets.append(abbyTarget)
  
        #trainingSessions2 = [item for sublist in trainingSessions for item in sublist] #FIX in oneSessionEncoder
        sessionsMinibatch = np.asarray(trainingSessions, dtype=theano.config.floatX)\
                                       .reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, dimIn))
        targetsMinibatch = np.asarray(trainingTargets, dtype=theano.config.floatX)

        predcostfun = predictFun(sessionsMinibatch)
    
        predtargets = predtargets + list(np.argmax(predcostfun,axis=1))
        actualtargets = actualtargets + list(np.argmax(targetsMinibatch, axis=1))
    
    #print "test accuracy: ", np.mean(np.argmax(predcostfun,axis=1) == np.argmax(targetsMinibatch, axis=1))
    #return trainingSessions
    return predtargets, actualtargets

In [163]:
predtar, acttar = predictClass(classifierPredict, hexSessions, comsDict, uniqIPs, hexDict,
                               numClasses, trainPercent, dimIn, maxPackets, packetTimeSteps,
                               padOldTimeSteps, batch_sizeClass)

In [164]:
def binaryPrecisionRecall(predictions, targets, numClasses = 4):
    for cla in range(numClasses):
        
        confustop = np.array([])
        confusbottom = np.array([])

        predictions = np.asarray(predictions).flatten()
        targets = np.asarray(targets).flatten()

        pred1 = np.where(predictions == cla)
        pred0 = np.where(predictions != cla)
        target1 = np.where(targets == cla)
        target0 = np.where(targets != cla)

        truePos = np.intersect1d(pred1[0],target1[0]).shape[0]
        trueNeg = np.intersect1d(pred0[0],target0[0]).shape[0]
        falsePos = np.intersect1d(pred1[0],target0[0]).shape[0]
        falseNeg = np.intersect1d(pred0[0],target1[0]).shape[0]

        top = np.append(confustop, (truePos, falsePos))
        bottom = np.append(confusbottom, (falseNeg, trueNeg))
        confusionMatrix = np.vstack((top, bottom))
        
        precision  = float(truePos)/(truePos + falsePos + 0.00001) #how much junk did we give user
        recall = float(truePos)/(truePos + falseNeg + 0.00001) #how much good stuff did we miss
        f1 = 2*((precision*recall)/(precision+recall))
        
        print 'class '+str(cla)+' precision: ', precision
        print 'class '+str(cla)+' recall:    ', recall
        print 'class '+str(cla)+' f1:        ', f1
        print
    
        #return confusionMatrix

In [165]:
binaryPrecisionRecall(predtar, acttar)

class 0 precision:  0.482122255108
class 0 recall:     0.981220634244
class 0 f1:         0.646558381337

class 1 precision:  0.999995000025
class 1 recall:     0.00442477866317
class 1 f1:         0.00881057229909

class 2 precision:  0.975446406798
class 2 recall:     0.997716872198
class 2 f1:         0.986455959674

class 3 precision:  0.999999977427
class 3 recall:     0.997747725276
class 3 f1:         0.998872581762



In [68]:
#[costClass, hEnc, hContext, pyx, softoutClass]
costfun[-1]

array([[ 0.25025687,  0.25071838,  0.25049034,  0.2485344 ],
       [ 0.25025731,  0.25071874,  0.25049034,  0.2485337 ],
       [ 0.25025731,  0.25071847,  0.25049055,  0.24853367],
       [ 0.25025856,  0.25071833,  0.25048986,  0.24853329],
       [ 0.25025856,  0.25071838,  0.2504898 ,  0.24853329],
       [ 0.25025845,  0.25071815,  0.25049001,  0.24853344],
       [ 0.25025776,  0.25071844,  0.25049028,  0.24853358],
       [ 0.250258  ,  0.25071803,  0.25049001,  0.24853392],
       [ 0.25025636,  0.25071862,  0.25049099,  0.24853413],
       [ 0.25025705,  0.25071841,  0.25049058,  0.24853401],
       [ 0.25025797,  0.250718  ,  0.25049022,  0.24853382],
       [ 0.25025809,  0.25071818,  0.25049016,  0.24853355],
       [ 0.25025845,  0.25071815,  0.25048983,  0.24853365],
       [ 0.25025812,  0.25071821,  0.25049016,  0.24853353],
       [ 0.25025803,  0.25071824,  0.25049031,  0.2485334 ],
       [ 0.25025859,  0.25071821,  0.2504898 ,  0.24853344],
       [ 0.25025752,  0.

In [ ]:
np.asarray(trainingSessions).shape

In [ ]:
np.argmax(classifierPredict(sessionsMinibatch),axis=1)

In [ ]:
np.argmax(targetsMinibatch, axis=1)

In [ ]:
adversaryList = [dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs), portDirSwitcher(sessionForEncoding), 
                 ipDirSwitcher(sessionForEncoding)]

abbyHexSession = random.sample(adversaryList, 1)[0]


In [ ]:
#testing for generalization

testCollect = []
for start, end in zip(range(trainIndex, len(hexSessionsKeys), batch_sizeClass/2),
                      range(trainIndex + batch_sizeClass/2, len(hexSessionsKeys), batch_sizeClass/2)):
    trainingTargets = []
    trainingSessions = []
    for trainKey in range(start, end):
        sessionForEncoding = hexSessions[hexSessions.keys()[trainKey]]

        #encode a normal session
        oneHotSes = oneSessionEncoder(sessionForEncoding,hexDict = hexDict, packetReverse=packetReverse, 
                                      padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets,
                                      packetTimeSteps = packetTimeSteps)
        trainingSessions.append(oneHotSes[0])
        trainingTargets.append(normalTarget)

        #encode an abby normal session
        abbyHexSession = dstIpSwapOut(sessionForEncoding, comsDict, uniqIPs) #95%, 88%
        #abbyHexSession = ipDirSwitcher(sessionForEncoding) #96%, 98%
        #abbyHexSession = portDirSwitcher(sessionForEncoding) #96%, 98%
        abbyOneHotSes = oneSessionEncoder(abbyHexSession,hexDict = hexDict,packetReverse=packetReverse, 
                                          padOldTimeSteps = padOldTimeSteps, maxPackets = maxPackets, 
                                          packetTimeSteps = packetTimeSteps)
        trainingSessions.append(abbyOneHotSes[0])
        trainingTargets.append(abbyTarget)

    #trainingSessions = [item for sublist in trainingSessions for item in sublist] #FIX in oneSessionEncoder
    sessionsMinibatch = np.asarray(trainingSessions, dtype=theano.config.floatX)\
                                   .reshape((batch_sizeClass*maxPackets, packetTimeSteps, 1, 257))
    targetsMinibatch = np.asarray(trainingTargets, dtype=theano.config.floatX)

    predcostfun = classifierPredict(sessionsMinibatch)
    testCollect.append(np.mean(np.argmax(predcostfun, axis=1) == targetsMinibatch[:,1]))

print 'test accuracy: ', np.mean(testCollect)
    


In [ ]:
len(testCollect)

In [ ]:
#[costClass,hEnc, hContext,pyx]
costfun[1][0][0] == costfun[1][1][0]

In [ ]:
classifierPredict(sessionsMinibatch[::-1])

In [ ]:
classifierPredict(sessionsMinibatch)

#Can be used with or without unsupervised learning

#X = T.matrix('classtest')
#Y = T.matrix('targets')
batch_sizeClass = 20
numClasses = 2
clippings = 5
learning_rateClass = 0.0001
classifierWts = IsotropicGaussian(0.03, 0)

bmlp = BatchNormalizedMLP(activations=[Logistic(),Logistic()], 
           dims=[dim, dim, numClasses],
           weights_init=classifierWts,
           biases_init=Constant(0.0001) )


bmlp.initialize()

data1, data2 = fork.apply(X) #data1 shape = (batch_size, maxPackets, dimIn)

if rnnType == 'gru':
    hEnc = rnn.apply(data1, data2) 
else:
    hEnc, _ = rnn.apply(data2, iterate = False)
    #hinit, _ = rnn.apply(data2)
    #hEnc = hinit[:,-1] #hEnc shape = (batch_size, maxPackets, dim) 

hEnc2 = T.reshape(hEnc[-1],(batch_sizeClass, maxPackets, dim))#the [:,-1] gets the last hidden state for 
                                                                #each obs in minibatch
                                                                #i.e. the last state for each sentence

data3, data4 = forkContext.apply(hEnc2)

if rnnType == 'gru':
    hContext = rnnContext.apply(data3, data4)
else:
    hinitContext, _ = rnnContext.apply(data4)
    hContext = hinitContext #hContext shape = (batch_size, maxPackets, dim)

data5, _ = forkDec.apply(hContext) 

#FIX bidirectional
'''if bidirectional:

    data3classRev = data3class[::-1]
    data4classRev = data4class[::-1]

    if rnnType == 'gru':
        hContextRev = rnnContextRev.apply(data3classRev, data4classRev)
    else:
        hinitContextRevClass, _ = rnnContextRev.apply(data4classRev)
        hContextRevClass = hinitContextRevClass

    hContextClass = T.concatenate((hContextClass, hContextRevClass), axis=2)'''
        

pyx = bmlp.apply(hContext[:,-1])
#softmax = Softmax()
#softoutClass = softmax.apply(pyx)
costClass = T.mean(CategoricalCrossEntropy().apply(Y, pyx))

cgClass = ComputationGraph([costClass])
paramsClass = VariableFilter(roles = [PARAMETER])(cgClass.variables)

updatesClass = Adam(paramsClass, costClass, learning_rateClass, c=clippings) 
#updatesClass = RMSprop(costClass, paramsClass, learning_rateClass, c=clippings)

#print 'grad compiling'
#gradients = T.grad(costClass, paramsClass)
#gradients = clip_norms(gradients, clippings)
#gradientFun = theano.function([X,Y], gradients, allow_input_downcast=True)
#print 'finish with grads'

print 'compiling functions you talented soul'
#classifierTrain = theano.function([X,Y], [costClass,hEnc, hEnc2, hContext,pyx], updates=updatesClass, allow_input_downcast=True)
#classifierPredict = theano.function([X], pyx, allow_input_downcast=True)
print 'finished compiling'